<h1 style='color:#0000ec'>8 &nbsp;Estudo comparativo - Soluções numéricas e analítica</h1>

<h2 style='color:#0000b7' >&nbsp; 8.1 &nbsp;Erro absoluto e percentual</h2>

&nbsp; &nbsp; &nbsp; Esta seção tem como objetivo comparar a solução analítica e a aproximação numérica com a população estimada para 2021 pelo IBGE.

&nbsp; &nbsp; &nbsp; Os resultados obtidos pela solução analítica, pelo método de Euler, Euler modificado e RK4:

<center><b>Aproximação populacional pelo modelo de Malthus</b></center>

| Método | Número de habitantes |
| --- | --- |
| LSODA | 209760.59999083 |
| Euler | 209759.35909435 |
| Euler modificado | 209759.42816025 |
|RK4 |209759.42816037 |
|RK45|  209760.60286601182|

<center>Fonte: O autor</center>

<br>

<center><b>Aproximação populacional pelo modelo de Verhulst</b></center>

| Método | Número de habitantes |
| --- | --- |
| LSODA | 208447.55940518 |
| Euler | 208446.62006659 |
| Euler modificado | 208446.56882837 |
|RK4 |208446.56882864 |
|RK45|  208447.55725928|

<center>Fonte: O autor</center>

&nbsp; &nbsp; &nbsp; Importando bibliotecas:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

&nbsp; &nbsp; &nbsp; Declarando o nome dos métodos, aproximações e a população estimada pelo IBGE:

In [ ]:
metodos = ['Analítico','LSODA','Euler','Euler modificado','RK4','RK45']

solucoes_malthus = [209761,
            209760.59999083,
            209759.35909435,
            209759.42816025,
            209759.42816037, 
            209760.60286601182]

solucoes_verhulst = [208447.5573077047,
            208447.55940518,
            208446.62006659,
            208446.56882837,
            208446.56882864, 
            208447.55725928]

index = [0,1,2,3,4]

solucao_IBGE = 212881

dados = [[0, 0, 0]]

&nbsp; &nbsp; &nbsp; A seguir, obtemos o erro das soluções obtidas com relação a população fornecida pelo IBGE.